In [13]:
import pandas as pd
import torch.nn as nn 
from transformers import AutoModel

In [5]:
from src.dataset import CommonLitDataset

In [6]:
df = pd.read_csv('./data/train.csv')
df.shape

(7165, 9)

In [7]:
train_df = df[df.prompt_id!='814d6b'].copy().reset_index(drop=True)
val_df = df[df.prompt_id=='814d6b'].copy().reset_index(drop=True)
train_df.shape, val_df.shape

((6062, 9), (1103, 9))

In [8]:
train_dataset = CommonLitDataset(model='distilbert-base-uncased', df=train_df)
val_dataset = CommonLitDataset(model='distilbert-base-uncased', df=val_df)

In [26]:
from transformers import AutoModel
import torch.nn as nn


class CommonLitModel(nn.Module):
    "CommonLitModel for CommonLit Readability Prize target prediction"

    def __init__(self, model_name, tok_len=30528) -> None:
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.model.resize_token_embeddings(tok_len)
        self.linear = nn.Linear(self.model.config.dim, 2)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask, **kwargs):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        output = self.linear(self.relu(output.last_hidden_state[:, 0, :]))
        return output


In [27]:
model = CommonLitModel('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
model(**train_dataset[0])

tensor([[0.0751, 0.0198]], grad_fn=<AddmmBackward0>)